[Prev](Primer04.ipynb)

DataJoint Primer. Section 5.
# Lookup tables

In [1]:
%matplotlib notebook
import datajoint as dj

DataJoint 0.2.1 (June 1, 2016)
Loading local settings from dj_local_conf.json


## Lookups store simple facts

In the [previous part](Primer04.ipynb) we have defined a new class that used the MySQL `enum` datatype. 

```python

@schema
class CorticalLayer(dj.Manual):
    definition = """
    # cortical layer a scan was performed in 
    -> expe.Scan
    ---
    layer    : enum("L1","L2/3","L4", "L5","L6")
    """
```

The reason why we used an `enum` and not a simple `char` field is that we wanted to standardize the names. We don't want other users to enter free text for this attribute that we would not know how to handle downstream in the analysis. However, enumerates have one drawback: Once the table is defined you cannot change its values. You need to redefine the table which means we have to play with MySQL code. An additional advantage is that `enum`s only work for one value. What if we want to predefine entire tuples?

These things can be easily achieved with lookup tables. The store simple facts like names or parameter settings, and we usually do not expect them to grow a lot. DataJoint provides a simple hook to fill the table with contents. 

Let's make a simple example by defining an experimenter table. That table could be used as an additional dependency in `expe.Session`. Because an experimenter is usually defined by more than one attribute and the number of experimenters can grow over time, a lookup table is a good choice. 

In [2]:
schema = dj.schema('dimitri_experiment',locals())

@schema
class Experimenter(dj.Lookup):
    definition = """
    # experimenter information
    
    experimenter_id    : tinyint # unique ID of the experimenter
    ---
    first_name         : varchar(128) # first name
    last_name          : varchar(128) # last name
    """
    
    contents = [
        (0, 'Dimitri','Yatsenko'),
        (1, 'Edgar','Walker'),
        (2, 'Jake','Reimer'),
        (3, 'Fabian','Sinz'),
    ]
    
Experimenter()

experimenter_id,first_name,last_name
0,Dimitri,Yatsenko
1,Edgar,Walker
2,Jake,Reimer
3,Fabian,Sinz


Let's take a closer look: Lookup tables inherit from `dj.Lookup`. This tier is very similar to `dj.Manual` except that the table names are generated a bit differently from the class names and that a lookup table can have a variable `contents` that predefine content that needs to be inserted. Contents only needs to be iterable and each item needs to be an insertable element (a tuple, a dictionary, or a record array). So this would work as well

```python
@schema
class Layer(dj.Lookup):
    definition = """
    # cortical layer
    layer     : char(5) # layer name
    ---
    """
    
    @property
    def contents(self):
        yield from zip(['L1', 'L2/3', 'L4', 'L5', 'L6'])
```

**Important:** `dj.Lookup` tables do not delete elements if you delete them from the iterable. They only add more elements and they only do this if the length of `contents` and the number of tuples in the relation do not match up. So if you replace an entry from contents by another one, nothing will happen. The reason for implementing it this way is simple efficiency. If would take a long time to match up all the contents everytime the python class object is built. Especially when working with Amazon AWS this can create quite some overhead. 


[Next](Primer06.ipynb)